In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, timezone
import json
import re

%run "./news_source/news.ipynb" import News

%run "./jgc_news/g1.ipynb" import G1
# %run "../jgc_news/dn.ipynb" import DN
# %run "../jgc_news/odia.ipynb" import ODia
# %run "../jgc_news/estadao.ipynb" import Estadao
# %run "../jgc_news/folha.ipynb" import Folha
# %run "../jgc_news/espn.ipynb" import Espn
# %run "../jgc_news/uol.ipynb" import Uol
# %run "../jgc_news/correio_braziliense.ipynb" import CorreioBraziliense

In [4]:
class GoogleAlertXml:

    def __init__(self, newsSource, sectionId, sectionName, sectionArr, tags):
        self.newsSource = newsSource
        self.sectionId = sectionId
        self.sectionName = sectionName
        self.sectionArr = sectionArr
        self.tags = tags
        
        self.xml_soup = self.get_xml()
        self.title_xml = self.get_title_xml()
        self.news_metadata = self.get_news_metadata()
        # self.source = self.get_source(self.newsSource)
        # self.hours_since_published = self.get_until_now()
        # self.newsData = self.get_news_data()


    def get_xml(self):
        browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
        xml_content = requests.get(self.newsSource, headers = browsers)
        soup = BeautifulSoup(xml_content.text, 'xml')
        return soup
    
    
    def get_title_xml(self):
        title = self.xml_soup.find('title').get_text()
        return title.split(sep=' - ')[1]
    
    
    def get_until_now(self):
        ts_str = self.xml_soup.find('published').get_text()
        date_time_obj = datetime.strptime(ts_str, '%Y-%m-%dT%H:%M:%SZ')
        date_time_obj = date_time_obj.replace(tzinfo=timezone.utc)
        current_time = datetime.now(timezone.utc)
        time_difference = current_time - date_time_obj
        hours_passed = time_difference.total_seconds() / 3600
        return hours_passed
    
    
    def get_source(self, url):
        if 'tntsports.com.br' in url:
            return 'tntsports'
        if 'correiobraziliense.com.br' in url:
            return 'correiobraziliense'
        if 'placar.com.br' in url:
            return 'placar'
        if 'flamengo.com.br' in url:
            return 'flamengo'
        if 'agoracomvoce.com.br' in url:
            return 'agoracomvoce'
        if 'folhape.com.br' in url:
            return 'folhape'
        if 'espn.com.br' in url:
            return 'espn'
        if 'uol.com.br' in url:
            return 'uol'
        if 'ge.globo.com' in url:
            return 'geglobo'
        return 'outro'





    def get_news_metadata(self):
        result = []
        news_all = self.xml_soup.find_all('entry')
        for news in news_all:
            title = news.find('title').get_text()
            url = news.find('link').get('href')
            source = self.get_source(url)
            timestamp = news.find('published').get_text()
            result += [News(source, url, title, timestamp)]
        return result






